In [300]:
# import required libraries

import numpy as np
import yfinance as yf
import pandas as pd
from datetime import date

In [301]:
# cleanup data converting string values to numeric values

def map_data(x):
    x[0] = x[0] / 1000
    x[4] = 1 if x[4].strip().lower() == 'yes' else 0
    x[5] = 1 if x[5].strip().lower() == 'yes' else 0
    x[6] = 1 if x[6].strip().lower() == 'yes' else 0
    x[7] = 1 if x[7].strip().lower() == 'yes' else 0
    x[8] = 1 if x[8].strip().lower() == 'yes' else 0
    x[10] = 1 if x[10].strip().lower() == 'yes' else 0
    x[11] = 1 if x[11].strip().lower() == 'furnished' else (2 if x[11].strip().lower() == 'semi-furnished' else 3)
    return x

In [302]:
# function to read data from csv and normalise it

def read_from_csv():
    data=pd.read_csv("housing.csv")
    
    # data.columns = data.columns.str.strip()
    # print(data.columns)
    
    X_orig = data[[
        'area',
        'bedrooms',
        'bathrooms',
        'stories',
        'mainroad',
        'guestroom',
        'basement',
        'hotwaterheating',
        'airconditioning',
        'parking',
        'prefarea',
        'furnishingstatus'
    ]].values.tolist()
    
    X = list(map(map_data, X_orig))
    
    Y = data["price"].values.tolist()

    return (X, Y)

In [303]:
# defining sigmoid function

def sigmoid(z):
    s = 1 / (1 + np.exp(-1 * z))
    return s

In [304]:
# initialise data structures

def init_data(X, Y):

    num_iterations=2
    learning_rate=0.009
    
    n_h = 4
    
    X = np.array(X)
    X = X.reshape(12, X.shape[0])
    
    Y = np.array(Y)
    Y = Y.reshape(1, Y.shape[0])
    
    W1 = np.random.randn(n_h, X.shape[0]) * .001
    W2 = np.random.randn(1, n_h) * .001
    b1 = np.random.randn(1, Y.shape[1]) * .001
    b2 = np.random.randn(1, Y.shape[1]) * .001
    
    m = X.shape[1]

    return (num_iterations, learning_rate, X, Y, W1, W2, b1, b2, m)

In [305]:
# get parameteres to predict values

(X, Y) = read_from_csv()

(num_iterations, learning_rate, X, Y, W1, W2, b1, b2, m) = init_data(X, Y)

for i in range(1, num_iterations):
    Z1 = np.dot(W1, X) + b1
    A1 = Z1
    Z2 = np.dot(W2, A1) + b2
    A2 = Z2
    
    dZ2 = A2 - Y
    dW2 = np.dot(dZ2, A1.T) / m
    db2 = np.sum(dZ2, axis=1, keepdims = True) / m

    dZ1 = np.dot(W2.T, dZ2) * (1 - np.power(A1, 2))
    dW1 = np.dot(dZ1, X.T) / m
    db1 = np.sum(dZ1, axis=1, keepdims = True) / m
    
    W1 = W1 - learning_rate * (np.power(dW1, 2) / dW1)
    b1 = b1 - learning_rate * (np.power(db1, 2) / db1)
    W2 = W2 - learning_rate * (np.power(dW2, 2) / dW2)
    b2 = b2 - learning_rate * (np.power(db2, 2) / db2)

In [310]:
def predict(values):
    values = np.array(map_data(values)).reshape(12, 1)
    return np.dot(W2, (np.dot(W1, values) + b1)) + b2

In [317]:
print(np.sum(predict([7420, 4, 2 , 3, 'Yes', 'No', 'No', 'No', 'Yes', 2, 'Yes' , 'Furnished']), axis=1))
print(np.sum(predict([8960, 4, 4 , 4, 'Yes', 'No', 'No', 'No', 'Yes', 3, 'No' , 'Furnished']), axis=1))

[13505902.2634684]
[11412500.24258532]
